# Activity 8: Re-training a model dynamically
In this activity, we re-train our model every time new data is available.

First, we start by importing `cryptonic`. Cryptonic is a simple software application developed for this course that implements all the steps up to this section using Python classes and modules. Consider Cryptonic a template on how you could develop similar applications.

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
import sys
import os

py_file_location = "/content/drive/My Drive/Deployment/Deployement"

sys.path.append(os.path.abspath(py_file_location))
from cryptonic import keys
from cryptonic import Model
from cryptonic import CoinMarketCap
from cryptonic.models import normalizations
import pandas as pd
from tqdm import tqdm_notebook
from datetime import datetime
%autosave 5
%matplotlib inline

ModuleNotFoundError: ignored

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Fetching Real-time Data
Throughout this project we have been using data originally provided by [CoinMarketCat](https://coinmarketcap.com/). We have created an interface for collecting both real-time and historical data as as part of `cryptonic`: the class `CoinMarketCap()`.

In [ ]:
print(CoinMarketCap())



        Crypto-currency data comes from the website CoinMarketCap.
        CoinMarketCap is can be accessed at: https://coinmarketcap.com/

        The permission to use the data is available on their FAQ

            https://coinmarketcap.com/faq/

        and reads:

            "Q: Am I allowed to use content (screenshots, data, graphs, etc.) 
            for one of my personal projects and/or commercial use?

            R: Absolutely! Feel free to use any content as you see fit. 
            We kindly ask that you cite us as a source."
        
        


In [ ]:
historic_data = pd.read_csv("/content/drive/MyDrive/Deployment/Deployement/bitcoin_cash_price.csv")
# historic_data.head()

historic_data['Date'] = historic_data['Date'].apply(lambda x: datetime.strptime(x, '%b %d, %Y').strftime('%Y-%m-%d'))
historic_data['Volume'] = historic_data['Volume'].apply(lambda x: None if x == '-' else x)
historic_data.columns = ['date', 'open', 'high', 'low', 'close', 'volume', 'market_cap']
historic_data

,date,open,high,low,close,volume,market_cap
0,2018-02-20,1543.27,1569.03,1414.35,1418.73,"820,947,000","26,199,800,000"
1,2018-02-19,1483.34,1553.81,1483.34,1534.77,"578,906,000","25,179,700,000"
2,2018-02-18,1552.10,1641.40,1428.49,1487.46,"907,873,000","26,344,200,000"
3,2018-02-17,1548.48,1568.64,1517.14,1551.39,"641,719,000","26,280,100,000"
4,2018-02-16,1373.16,1558.66,1369.68,1552.20,"961,010,000","23,302,000,000"
...,...,...,...,...,...,...,...
208,2017-07-27,417.10,460.97,367.78,385.48,"533,207",-
209,2017-07-26,407.08,486.16,321.79,365.82,"1,784,640",-
210,2017-07-25,441.35,541.66,338.09,406.90,"524,908",-
211,2017-07-24,412.58,578.89,409.21,440.70,"190,952",-


Our model is designed to work with daily data. Let's go ahead and collect historic daily data from CoinMarketCap (this is the same data used previously).

In [ ]:
# historic_data = CoinMarketCap.historic()
# historic_data.head()
# GOT DATA USING BINANCE API , CANNOT SHARES KEYS FILE


The data contains practically the same variables from our earlier dataset. However, much of the data comes from an earlier period. Recent Bitcoin prices have gained a lot of volatility if compared to the prices of a few years ago. Before using this data in our model, let's make sure to filter it to dates after January 1, 2017.

In [ ]:
#
#  Using the Pandas API, filter the dataframe
#  for observations from 2017 only. 
# 
#  Hint: use the `date` variable.
#
model_data = historic_data[historic_data['date'] > "2017-08-01"]
model_data


,date,open,high,low,close,volume,market_cap
0,2018-02-20,1543.27,1569.03,1414.35,1418.73,"820,947,000","26,199,800,000"
1,2018-02-19,1483.34,1553.81,1483.34,1534.77,"578,906,000","25,179,700,000"
2,2018-02-18,1552.10,1641.40,1428.49,1487.46,"907,873,000","26,344,200,000"
3,2018-02-17,1548.48,1568.64,1517.14,1551.39,"641,719,000","26,280,100,000"
4,2018-02-16,1373.16,1558.66,1369.68,1552.20,"961,010,000","23,302,000,000"
...,...,...,...,...,...,...,...
198,2017-08-06,212.18,223.70,203.44,220.66,"107,606,000","3,497,290,000"
199,2017-08-05,231.11,273.04,200.98,213.15,"144,043,000","3,809,330,000"
200,2017-08-04,362.18,386.93,233.05,233.05,"185,038,000","5,969,720,000"
201,2017-08-03,448.49,519.28,364.05,364.05,"161,518,000","7,392,030,000"


## The `Model()` Class
We have also created the class `Model()` which compiles all the code we have written so far. We will use that class to build, train, and evaluate our model.

In [ ]:
M = Model(data=model_data,
          variable='close',
          predicted_period_size=7)

In [ ]:
M.build()

In [ ]:
M.train(epochs=100, verbose=1)

Epoch 1/100
1/1 [==============================] - 0s 1ms/step - loss: 0.0725
Epoch 2/100
1/1 [==============================] - 0s 1ms/step - loss: 0.0693
Epoch 3/100
1/1 [==============================] - 0s 1ms/step - loss: 0.0670
Epoch 4/100
1/1 [==============================] - 0s 1ms/step - loss: 0.0652
Epoch 5/100
1/1 [==============================] - 0s 1ms/step - loss: 0.0635
Epoch 6/100
1/1 [==============================] - 0s 1ms/step - loss: 0.0619
Epoch 7/100
1/1 [==============================] - 0s 1ms/step - loss: 0.0605
Epoch 8/100
1/1 [==============================] - 0s 1ms/step - loss: 0.0591
Epoch 9/100
1/1 [==============================] - 0s 1ms/step - loss: 0.0578
Epoch 10/100
1/1 [==============================] - 0s 3ms/step - loss: 0.0565
Epoch 11/100
1/1 [==============================] - 0s 2ms/step - loss: 0.0552
Epoch 12/100
1/1 [==============================] - 0s 2ms/step - loss: 0.0540
Epoch 13/100
1/1 [==============================] - 0s 1ms/st

We can now use the model for making predictions with the `predict()` method. The parameter `denormalized` will return values in the original scale of the data. In our case, US dollars.

In [ ]:
M.predict(denormalized=True)

array([1415.1149 , 1313.3135 ,  909.54565,  867.5113 ,  949.6065 ,
       1485.8989 , 1854.0914 ], dtype=float32)

We now evaluate our model to inspect the statistics for the last epoch of training compared to a single test week.

In [ ]:
M.evaluate()

{'mape': 21.58, 'mse': 0.0, 'rmse': 373.03}

Finally, we can now save the trained model on disk for later use.

In [ ]:
M.save('bitcoin_model_prod_v0.h5')

Our `Model()` class can also load a previously trained model when instantiated with the `path` parameter.

In [ ]:
M = Model(path='bitcoin_model_prod_v0.h5',
          data=model_data,
          variable='close',
          predicted_period_size=7)

In [ ]:
M.predict(denormalized=True)

array([1415.1149 , 1313.3135 ,  909.54565,  867.5113 ,  949.6065 ,
       1485.8989 , 1854.0914 ], dtype=float32)

## New Data, Re-train Old Model
One strategy discussed earlier regards the re-training of our model with new data. In our case, our biggest concern is to shape data in a way that the model has been configured. As an example, we will configure our model to predict a week using 40 weeks. We will first train the model with the first 40 weeks of 2017, then continue to re-train it over the following weeks until we reach week 50.

First, let's build a model with the first set of data. Notice how we use `7*40 + 7` as the indexer. This is because we use 40 weeks for training and 1 week for testing. 

In [ ]:
M1 = Model(data=model_data[0*7:7*40 + 7],
          variable='close',
          predicted_period_size=7)

In [ ]:
M1.build()

In [ ]:
M1.train()

In [ ]:
#
#  Complete the range function and
#  the model_data filtering parameters
#  using an index to split the data in overlapping
#  groups of 7 days. Then, re-train our model
#  and collect the results.
#
#  The variables A, B, C, and D are placeholders.
#
A = 2
B = 41
C = 3
D = 42
results = []
for i in range(A, B):
    M1.train(model_data[C:D])
    results.append(M.evaluate())

In [ ]:
for i, result in enumerate(results):
    print(f'Week {40+i+1}: {result}')

Week 41: {'mse': 0.0, 'rmse': 373.03, 'mape': 21.58}
Week 42: {'mse': 0.0, 'rmse': 373.03, 'mape': 21.58}
Week 43: {'mse': 0.0, 'rmse': 373.03, 'mape': 21.58}
Week 44: {'mse': 0.0, 'rmse': 373.03, 'mape': 21.58}
Week 45: {'mse': 0.0, 'rmse': 373.03, 'mape': 21.58}
Week 46: {'mse': 0.0, 'rmse': 373.03, 'mape': 21.58}
Week 47: {'mse': 0.0, 'rmse': 373.03, 'mape': 21.58}
Week 48: {'mse': 0.0, 'rmse': 373.03, 'mape': 21.58}
Week 49: {'mse': 0.0, 'rmse': 373.03, 'mape': 21.58}
Week 50: {'mse': 0.0, 'rmse': 373.03, 'mape': 21.58}
Week 51: {'mse': 0.0, 'rmse': 373.03, 'mape': 21.58}
Week 52: {'mse': 0.0, 'rmse': 373.03, 'mape': 21.58}
Week 53: {'mse': 0.0, 'rmse': 373.03, 'mape': 21.58}
Week 54: {'mse': 0.0, 'rmse': 373.03, 'mape': 21.58}
Week 55: {'mse': 0.0, 'rmse': 373.03, 'mape': 21.58}
Week 56: {'mse': 0.0, 'rmse': 373.03, 'mape': 21.58}
Week 57: {'mse': 0.0, 'rmse': 373.03, 'mape': 21.58}
Week 58: {'mse': 0.0, 'rmse': 373.03, 'mape': 21.58}
Week 59: {'mse': 0.0, 'rmse': 373.03, 'mape': 

In [ ]:
M1.predict(denormalized=True)

array([1553.3234, 1351.5917, 1342.5005, 1334.2539, 1353.6796, 1834.5966,
       1947.9082], dtype=float32)

## New Data, New Model
Another strategy is to create and train a new model evey time new data is available. This approach tends to reduce catastrophic forgetting, but training time increases as data increases. 

It's implementation is quite simple.

Let's assume we have old data for 49 weeks of 2017 and after a week we now have new data. We represent this wtih the variables `old_data` and `new_data`. 

In [ ]:
old_data = model_data[0*7:7*48 + 7]

In [ ]:
new_data = model_data[0*7:7*49 + 7]

In [ ]:
M2 = Model(data=old_data,
          variable='close',
          predicted_period_size=7)

In [ ]:
M2.build()
M1.train()

In [ ]:
M2.predict(denormalized=True)

array([1400.6511, 1324.7882, 1404.8326, 1377.3014, 1403.285 , 1446.8893,
       1373.4232], dtype=float32)

Now, assume that new data is available. Using this technique we go ahead and create a new model using only the new data. 

In [ ]:
#
#  Re-instantiate the model with the Model()
#  class using the new_data variable instead
#  of the old_data one. 
#
M3 = Model(data=new_data,
          variable='close',
          predicted_period_size=7)

In [ ]:
M3.build()
M3.train()

In [ ]:
M3.predict(denormalized=True)

array([1412.37   , 1308.8905 ,  914.1812 ,  880.5174 ,  964.04956,
       1498.3904 , 1851.972  ], dtype=float32)

This approach is very simple to implement and tends to work well. We will be using this to deploy our application.